In [1]:
import nltk
#Необходимо прогнать эти строки при первом запуске
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

import pymorphy2
import codecs
import string
import re

from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
analyzer = pymorphy2.MorphAnalyzer()



# список символов, которые удаются из текста
MARKS = [',', '.', ':', '?', '«', '»', '-', '(', ')', '!', '\'', "—", ';', "”", "...", "\'\'", "/**//**/",
         "“", "„", "–"]

In [5]:
def parse_words_from_html(file):
    html_page = codecs.open(f"D:/Documents/GitHub/WebParser/src/pages/{file}", 'r', 'utf-8')
    
    soup = BeautifulSoup(html_page.read(), features='html.parser')
    # kill all script, style, meta, links, span, a, time, button, li, dt, h2, h3, legend elements
    for script in soup(
            ["script", "style", "meta", "link", "span", "a", "time", "button", "li", "dt", "h2", "h3", "legend"]):
        script.extract()  # rip it out

    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    # split to words. Тут пока со знаками препинания.
    return text.split()

def tokenize():
    words = []
    with open("D:/Documents/GitHub/WebParser/src/pages/index.txt", "r") as index:
        lines = index.readlines()
        numbers = [line[: line.find(" ")] for line in lines]
        for num in numbers:
            words.extend(parse_words_from_html(f"{num}.txt"))
        with open("words.txt", "w", encoding="utf-8") as file:
            filtered_words = filter_words(words)
            for word in filtered_words:
                file.write(word + '\n')
                
def filter_words(words):
    words = list(filter(lambda word: (word not in string.punctuation) and (word not in MARKS), words))
    filtered_words = []
    for word in words:
        filtered_words.append("".join(filter(lambda char: char not in MARKS, word)))
    stop_words_rus = set(stopwords.words('russian'))
    stop_words_eng = set(stopwords.words('english'))
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_rus]
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_eng]
    filtered_words = filter(lambda word: word.isalpha(), filtered_words)
    return filtered_words

def lemmatize():
    # считываем все слова из ранее полученного файла
    with open("words.txt", "r", encoding="utf-8") as words_list:
        words = words_list.readlines()
    # словарь, где ключ - начальная форма слова, а значение - список вариаций этого слова, которые встретились в данных
    lem_dict = {}
        
    rus_filter = re.compile("[а-яА-Я]+")
    eng_filter = re.compile("[a-zA-Z]+")
    rus_words = [w.strip() for w in filter(rus_filter.match, words)]
    eng_words = [w.strip() for w in filter(eng_filter.match, words)]
    for word in rus_words:
        # получаем начальную форму слова
        normal_form = get_normal_form_rus(word)
        if normal_form:
            # если такое слово еще не встречалось,создаем ключ с нормальной формой и помещаем само слово как значение
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word.strip()]
            # если такое слово встречалось ранее, добавляем его вариацию в список значений
            else:
                if(word not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word.strip())
    for word in eng_words:
        # получаем начальную форму слова
        normal_form = get_normal_form_eng(word)
        if normal_form:
            # если такое слово еще не встречалось,создаем ключ с нормальной формой и помещаем само слово как значение
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word.strip()]
            # если такое слово встречалось ранее, добавляем его вариацию в список значений
            else:
                if(word not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word.strip())
    print(lem_dict)

    # записываем полученные результаты в формате:
    # "начальная форма слова: токен токен ..."
    # знак ":" служит разделителем между ключом и значениями
    file = open("lemmatized_tokens.txt", "w", encoding="utf-8")
    for word, tokens in lem_dict.items():
        file.write(f"{word}:")
        [file.write(f" {tok}") for tok in set(tokens)]
        file.write("\n")
    file.close()

def get_normal_form_rus(word):
    return analyzer.parse(word)[0].normal_form
    

def get_normal_form_eng(word):
    return lemmatizer.lemmatize(word)
    

In [3]:
tokenize()

In [6]:
lemmatize()

{'учебник': ['учебник', 'учебниках', 'учебников', 'учебники'], 'начинающий': ['начинающих'], 'продвинуть': ['продвинутых', 'продвинутые', 'продвинутыми'], 'поиск': ['поиск', 'поиски', 'поиска', 'поиске'], 'присоединяться': ['присоединяйтесь'], 'мы': ['нам', 'мы', 'нами', 'нас'], 'различный': ['различным', 'различные', 'различными', 'различных', 'различной', 'различного'], 'тематика': ['тематикам'], 'дмитрий': ['дмитрий'], 'виноградов': ['виноградов'], 'подробный': ['подробнее', 'подробное', 'подробней', 'подробный', 'подробную', 'подробные'], 'никакой': ['никакие', 'никакого', 'никаких'], 'другой': ['другие', 'другими', 'другим', 'другое', 'другую', 'другому', 'других', 'другого', 'другая'], 'человек': ['люди', 'человек', 'людей'], 'иметь': ['имеют', 'имеет', 'имеем', 'иметь'], 'этот': ['этому', 'этих', 'этот', 'эта', 'эту', 'этими', 'эти', 'этой'], 'сайт': ['сайту', 'сайта', 'сайте', 'сайт', 'сайтам', 'сайтом'], 'отношение': ['отношения', 'отношений'], 'просто': ['просто'], 'заниматьс